In [171]:
import os
import json
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from PIL import Image


In [172]:

# 경로 설정
training_image_path = "/home/gyuha_lee/DCC2024/dataset/bg_remove/rembg/90/training_image_no_bg"
validation_image_path = "/home/gyuha_lee/DCC2024/dataset/bg_remove/rembg/90/validation_image_no_bg"
model_path = "/home/gyuha_lee/DCC2024/mission1/WITHOUT_PRETRAINED/resnet18_gender_style_pretrained.pth"  # 1-2에서 학습된 모델 가중치 파일 경로
json_path = "/home/gyuha_lee/DCC2024/mission2/top_100_preference.json"  # 2-2에서 생성된 CSV 파일 경로


In [173]:

# 1. ResNet-18 모델 로드 및 학습된 가중치 불러오기
model = models.resnet18(pretrained=False)  # 학습된 가중치를 로드할 것이므로 pretrained=False 설정
model.fc = nn.Identity()  # 마지막 FC 레이어를 제거하여 특징 벡터를 추출하도록 설정
model.load_state_dict(torch.load(model_path), strict=False)  # strict=False로 불필요한 키 무시하고 가중치 불러오기
model.eval()  # 평가 모드로 전환

/home/gyuha_lee/miniconda3/envs/dcc2024/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gyuha_lee/miniconda3/envs/dcc2024/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_65014/291212598.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [174]:

# 2. 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [175]:

# 3. 특징 벡터 추출 함수 정의(끝에서 두번째)
def extract_feature_vector(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원 추가
    with torch.no_grad():
        feature_vector = model(image).squeeze().numpy()
    return feature_vector


In [176]:
with open(json_path, 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)

In [177]:
# 5. 사용자별 평균 벡터 계산 및 Validation 데이터 유사도 비교
results = []

total_users = len(data)  # 총 사용자 수
user_counter = 0  # 사용자 진행 카운터

for user_id, user_data in data.items():
    user_counter += 1
    print(f"Processing user {user_counter}/{total_users} (ID: {user_id})...")  # 진행 현황 출력

    # Training 데이터에서 선호 및 비선호 파일 리스트 추출
    training_preferred_files = user_data['Training']['선호']
    training_non_preferred_files = user_data['Training']['비선호']
    validation_preferred_files = user_data['Validation']['선호']
    validation_non_preferred_files = user_data['Validation']['비선호']
    
    # Training 선호 및 비선호 파일의 특징 벡터 추출 및 평균 계산
    preferred_features = [extract_feature_vector(os.path.join(training_image_path, img)) for img in training_preferred_files if img]
    non_preferred_features = [extract_feature_vector(os.path.join(training_image_path, img)) for img in training_non_preferred_files if img]
    
    average_preferred_vector = np.mean(preferred_features, axis=0) if preferred_features else None
    average_non_preferred_vector = np.mean(non_preferred_features, axis=0) if non_preferred_features else None

    # Validation 데이터의 특징 벡터 추출 및 유사도 비교
    for val_img in validation_preferred_files:
        if val_img:  # 빈 문자열 또는 None 방지
            val_feature = extract_feature_vector(os.path.join(validation_image_path, val_img))
            if average_preferred_vector is not None:
                similarity = cosine_similarity(val_feature.reshape(1, -1), average_preferred_vector.reshape(1, -1))[0][0]
                predicted_label = 1 if similarity >= 0.7 else 0  # 유사도가 0.7 이상이면 선호로 예측
                true_label = 1  # 실제 레이블은 선호
                result = 'O' if true_label == predicted_label else 'X'
                
                # 결과 추가
                results.append({
                    'user_id': user_id,
                    'validation_file': val_img,
                    'true_label': true_label,
                    'predicted_label': predicted_label,
                    'result': result,
                    'preference_type': '선호'
                })
                
                # 진행 현황 출력
                print(f"Validation (선호): {val_img}, Similarity: {similarity:.2f}, Predicted: {'선호' if predicted_label == 1 else '비선호'}, Result: {result}")

    for val_img in validation_non_preferred_files:
        if val_img:  # 빈 문자열 또는 None 방지
            val_feature = extract_feature_vector(os.path.join(validation_image_path, val_img))
            if average_non_preferred_vector is not None:
                similarity = cosine_similarity(val_feature.reshape(1, -1), average_non_preferred_vector.reshape(1, -1))[0][0]
                predicted_label = 1 if similarity >= 0.7 else 0  # 유사도가 0.7 이상이면 선호로 예측
                true_label = 0  # 실제 레이블은 비선호
                result = 'O' if true_label == predicted_label else 'X'
                
                # 결과 추가
                results.append({
                    'user_id': user_id,
                    'validation_file': val_img,
                    'true_label': true_label,
                    'predicted_label': predicted_label,
                    'result': result,
                    'preference_type': '비선호'
                })
                
                # 진행 현황 출력
                print(f"Validation (비선호): {val_img}, Similarity: {similarity:.2f}, Predicted: {'선호' if predicted_label == 1 else '비선호'}, Result: {result}")

print("Processing completed for all users.")  # 전체 처리 완료 메시지 출력


Processing user 1/100 (ID: 64747)...
Validation (선호): W_46907_80_powersuit_W.jpg, Similarity: 0.84, Predicted: 선호, Result: O
Validation (선호): W_44330_10_sportivecasual_W.jpg, Similarity: 0.82, Predicted: 선호, Result: O
Validation (선호): W_39164_00_oriental_W.jpg, Similarity: 0.79, Predicted: 선호, Result: O
Validation (선호): W_37491_70_military_W.jpg, Similarity: 0.84, Predicted: 선호, Result: O
Validation (선호): W_20598_70_military_W.jpg, Similarity: 0.83, Predicted: 선호, Result: O
Validation (선호): W_30988_90_kitsch_W.jpg, Similarity: 0.90, Predicted: 선호, Result: O
Validation (선호): W_38588_19_genderless_W.jpg, Similarity: 0.82, Predicted: 선호, Result: O
Validation (선호): W_22510_80_powersuit_W.jpg, Similarity: 0.86, Predicted: 선호, Result: O
Validation (선호): W_05628_00_cityglam_W.jpg, Similarity: 0.75, Predicted: 선호, Result: O
Validation (비선호): W_34024_10_sportivecasual_W.jpg, Similarity: 0.85, Predicted: 선호, Result: X
Validation (비선호): W_14102_50_feminine_W.jpg, Similarity: 0.90, Predicted: 선호, 

In [178]:
# 모든 행과 열을 출력하도록 설정
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.width', None)  # 출력의 너비를 화면에 맞추기
pd.set_option('display.max_colwidth', None)  # 각 열의 최대 너비를 None으로 설정

In [179]:
# 6. 예측 결과 리스트 사용
results_list = results  # `results`는 이미 리스트 형태이므로 그대로 사용


In [180]:

# 7. 사용자별 예측 결과를 JSON 형식으로 정리
organized_results = {}

# 3-2의 예측 결과를 각 사용자별로 정리
for result in results_list:
    # 각 항목에 필요한 키가 모두 있는지 확인
    required_keys = ['user_id', 'validation_file', 'predicted_label', 'result', 'preference_type']
    if not all(key in result for key in required_keys):
        raise KeyError(f"One of the required keys is missing in the result: {result}")

    user_id = result['user_id']

    if user_id not in organized_results:
        organized_results[user_id] = {
            "Validation": {
                "선호": [],
                "비선호": []
            }
        }

    result_entry = {
        "파일명": result['validation_file'],
        "예측 결과": "선호" if result['predicted_label'] == 1 else "비선호",
        "결과": result['result']
    }

    if result['preference_type'] == "선호":
        organized_results[user_id]["Validation"]["선호"].append(result_entry)
    elif result['preference_type'] == "비선호":
        organized_results[user_id]["Validation"]["비선호"].append(result_entry)

In [181]:
# JSON 파일로 저장 (정리된 데이터를 저장)
json_output_path = "/home/gyuha_lee/DCC2024/mission3/organized_resuls.json"
with open(json_output_path, 'w', encoding='utf-8') as f:
    json.dump(organized_results, f, ensure_ascii=False, indent=4)
print("Organized prediction results saved to JSON.")

Organized prediction results saved to JSON.


In [182]:
# 8. 사용자별 예측 결과 출력
top_100_users = list(organized_results.keys())[:100]
organized_output = []

for user_id in top_100_users:
    validation_preferred_results = [
        f"{result['파일명']} (예측: {result['예측 결과']}, 결과: {result['결과']})"
        for result in organized_results[user_id]["Validation"]["선호"]
    ]
    validation_preferred = '\n'.join(validation_preferred_results)  # 줄바꿈하여 출력

    validation_non_preferred_results = [
        f"{result['파일명']} (예측: {result['예측 결과']}, 결과: {result['결과']})"
        for result in organized_results[user_id]["Validation"]["비선호"]
    ]
    validation_non_preferred = '\n'.join(validation_non_preferred_results)  # 줄바꿈하여 출력

    # 응답자별 데이터를 리스트에 추가
    organized_output.append([
        user_id,
        validation_preferred,
        validation_non_preferred
    ])

In [183]:
# Pandas 데이터프레임 생성 및 출력
organized_df = pd.DataFrame(organized_output, columns=[
    '응답자 ID', 
    'Validation 선호 파일 예측 결과', 
    'Validation 비선호 파일 예측 결과'
])

In [184]:
# 인덱스를 1부터 시작하게 설정
organized_df.index = pd.RangeIndex(start=1, stop=len(organized_df) + 1, step=1)

# 데이터프레임 출력
print("----- Organized Prediction Results for Top 100 Users -----")
organized_df.head(100)

----- Organized Prediction Results for Top 100 Users -----


,응답자 ID,Validation 선호 파일 예측 결과,Validation 비선호 파일 예측 결과
1,64747,"W_46907_80_powersuit_W.jpg (예측: 선호, 결과: O)\nW_44330_10_sportivecasual_W.jpg (예측: 선호, 결과: O)\nW_39164_00_oriental_W.jpg (예측: 선호, 결과: O)\nW_37491_70_military_W.jpg (예측: 선호, 결과: O)\nW_20598_70_military_W.jpg (예측: 선호, 결과: O)\nW_30988_90_kitsch_W.jpg (예측: 선호, 결과: O)\nW_38588_19_genderless_W.jpg (예측: 선호, 결과: O)\nW_22510_80_powersuit_W.jpg (예측: 선호, 결과: O)\nW_05628_00_cityglam_W.jpg (예측: 선호, 결과: O)","W_34024_10_sportivecasual_W.jpg (예측: 선호, 결과: X)\nW_14102_50_feminine_W.jpg (예측: 선호, 결과: X)\nW_47169_70_hippie_W.jpg (예측: 선호, 결과: X)\nW_02498_50_feminine_W.jpg (예측: 선호, 결과: X)\nW_11610_90_grunge_W.jpg (예측: 선호, 결과: X)\nW_27828_60_minimal_W.jpg (예측: 선호, 결과: X)"
2,63405,"W_01853_60_mods_M.jpg (예측: 선호, 결과: O)\nW_15294_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_02879_90_hiphop_M.jpg (예측: 선호, 결과: O)\nW_02677_60_mods_M.jpg (예측: 선호, 결과: O)\nW_06860_19_normcore_M.jpg (예측: 선호, 결과: O)\nW_04684_90_hiphop_M.jpg (예측: 선호, 결과: O)\nW_04522_50_ivy_M.jpg (예측: 선호, 결과: O)","W_16755_00_metrosexual_M.jpg (예측: 선호, 결과: X)\nW_12304_80_bold_M.jpg (예측: 선호, 결과: X)\nW_12904_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_17443_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_16501_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_07187_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_15140_80_bold_M.jpg (예측: 선호, 결과: X)"
3,64346,"W_07316_00_metrosexual_M.jpg (예측: 선호, 결과: O)\nW_24103_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_29990_90_hiphop_M.jpg (예측: 선호, 결과: O)\nW_09154_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_29918_19_normcore_M.jpg (예측: 선호, 결과: O)","W_16430_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_16121_80_bold_M.jpg (예측: 선호, 결과: X)\nW_24250_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_26099_19_normcore_M.jpg (예측: 선호, 결과: X)\nW_24838_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_24931_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_00496_60_mods_M.jpg (예측: 선호, 결과: X)"
4,64561,"W_35091_80_powersuit_W.jpg (예측: 선호, 결과: O)\nW_18205_50_feminine_W.jpg (예측: 선호, 결과: O)\nW_41448_10_sportivecasual_W.jpg (예측: 선호, 결과: O)\nW_33305_60_space_W.jpg (예측: 선호, 결과: O)\nW_30671_70_hippie_W.jpg (예측: 선호, 결과: O)\nW_06046_10_sportivecasual_W.jpg (예측: 선호, 결과: O)\nW_22239_60_space_W.jpg (예측: 선호, 결과: O)\nW_38656_10_sportivecasual_W.jpg (예측: 선호, 결과: O)","W_48457_60_minimal_W.jpg (예측: 선호, 결과: X)\nW_22943_10_athleisure_W.jpg (예측: 선호, 결과: X)\nW_33240_80_bodyconscious_W.jpg (예측: 선호, 결과: X)\nW_23519_60_minimal_W.jpg (예측: 선호, 결과: X)"
5,65139,"W_63644_10_sportivecasual_M.jpg (예측: 선호, 결과: O)","W_29942_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_58793_00_metrosexual_M.jpg (예측: 선호, 결과: X)\nW_24717_60_mods_M.jpg (예측: 선호, 결과: X)\nW_24517_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_52693_00_metrosexual_M.jpg (예측: 선호, 결과: X)\nW_54129_19_normcore_M.jpg (예측: 선호, 결과: X)\nW_31913_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_28314_10_sportivecasual_M.jpg (예측: 선호, 결과: X)\nW_51514_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_54465_80_bold_M.jpg (예측: 선호, 결과: X)\nW_27138_60_mods_M.jpg (예측: 선호, 결과: X)"
6,66513,"W_14828_50_classic_W.jpg (예측: 선호, 결과: O)","W_56334_10_sportivecasual_W.jpg (예측: 선호, 결과: X)\nW_60553_00_cityglam_W.jpg (예측: 선호, 결과: X)\nT_06910_50_classic_W.jpg (예측: 선호, 결과: X)\nW_39793_80_powersuit_W.jpg (예측: 선호, 결과: X)\nW_37404_60_space_W.jpg (예측: 선호, 결과: X)\nW_14914_50_feminine_W.jpg (예측: 선호, 결과: X)\nW_53112_90_lingerie_W.jpg (예측: 선호, 결과: X)\nW_44520_70_punk_W.jpg (예측: 선호, 결과: X)\nW_10984_50_feminine_W.jpg (예측: 선호, 결과: X)"
7,59704,"W_16219_70_hippie_M.jpg (예측: 선호, 결과: O)\nW_15244_80_bold_M.jpg (예측: 선호, 결과: O)\nW_01853_60_mods_M.jpg (예측: 선호, 결과: O)\nW_01549_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_04636_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_12092_80_bold_M.jpg (예측: 선호, 결과: O)\nW_02728_60_mods_M.jpg (예측: 선호, 결과: O)","W_19833_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_17697_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_12476_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_15120_60_mods_M.jpg (예측: 선호, 결과: X)\nW_06875_90_hiphop_M.jpg (예측: 선호, 결과: X)"
8,60173,"W_14570_60_minimal_W.jpg (예측: 선호, 결과: O)\nW_00152_50_feminine_W.jpg (예측: 선호, 결과: O)\nW_06015_80_powersuit_W.jpg (예측: 선호, 결과: O)","W_01236_10_sportivecasual_W.jpg (예측: 선호, 결과: X)\nW_14299_70_disco_W.jpg (예측: 선호, 결과: X)\nW_0

In [185]:
# 6. 예측 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 7. 성능 평가
true_labels = results_df['true_label']
predicted_labels = results_df['predicted_label']

# 정확도, 정밀도, 재현율, F1 점수 계산
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# 성능 결과 출력
print("----- Model Performance Metrics -----")
print(f"Accuracy (정확도): {accuracy:.2f}")
print(f"Precision (정밀도): {precision:.2f}")
print(f"Recall (재현율): {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("-------------------------------------")

----- Model Performance Metrics -----
Accuracy (정확도): 0.40
Precision (정밀도): 0.40
Recall (재현율): 1.00
F1 Score: 0.57
-------------------------------------
